In [6]:
import os
import fitz       # PyMuPDF
import pandas as pd
from PIL import Image
from io import BytesIO
from tqdm import tqdm
from docx2pdf import convert

def preprocess_docs(input_dir: str):
    """
    Convert all .docx/.doc files in input_dir into PDFs
    with the same base filename.
    """
    for filename in os.listdir(input_dir):
        lower = filename.lower()
        if lower.endswith((".docx", ".doc")):
            doc_path = os.path.join(input_dir, filename)
            pdf_path = os.path.join(
                input_dir,
                os.path.splitext(filename)[0] + ".pdf"
            )
            try:
                print(f"🔧 Converting {filename} → {os.path.basename(pdf_path)} …")
                convert(doc_path, pdf_path)
            except Exception as e:
                print(f"⚠️ Failed to convert {filename}: {e}")

def extract_images_from_pdfs(
    input_dir: str,
    output_dir: str,
    min_width: int = 1200,
    metadata_filename: str = "image_metadata.csv"
) -> pd.DataFrame:
    """
    1. Convert any .doc/.docx to .pdf.
    2. Walk through each PDF in input_dir, extract all embedded images,
    3. Save images under output_dir/<pdf_basename>/…,
    4. Compile a CSV of metadata (file, page, dimensions, resolution, high-res flag).
    Returns the metadata DataFrame.
    """
    # Step 1: convert Word docs
    preprocess_docs(input_dir)

    os.makedirs(output_dir, exist_ok=True)
    records = []

    for filename in os.listdir(input_dir):
        if not filename.lower().endswith(".pdf"):
            continue

        pdf_path    = os.path.join(input_dir, filename)
        base_name   = os.path.splitext(filename)[0]
        doc         = fitz.open(pdf_path)

        # Create subfolder for this PDF’s images
        pdf_img_dir = os.path.join(output_dir, base_name)
        os.makedirs(pdf_img_dir, exist_ok=True)

        print(f"\n📄 Extracting images from {filename} ({len(doc)} pages)…")
        for page_idx in tqdm(range(len(doc)), desc="Pages", leave=False):
            page   = doc[page_idx]
            images = page.get_images(full=True)
            if not images:
                continue

            for img_idx, img_info in enumerate(images, start=1):
                xref      = img_info[0]
                img_data  = doc.extract_image(xref)
                img_bytes = img_data["image"]

                try:
                    img = Image.open(BytesIO(img_bytes))
                except Exception as e:
                    print(f" ⚠️ Couldn’t open image on page {page_idx+1}, idx {img_idx}: {e}")
                    continue

                w, h     = img.size
                img_name = f"{base_name}_p{page_idx+1}_img{img_idx}.png"
                img_path = os.path.join(pdf_img_dir, img_name)
                img.save(img_path)

                records.append({
                    "pdf_file":    filename,
                    "page":        page_idx + 1,
                    "image_name":  img_name,
                    "image_path":  img_path,
                    "width":       w,
                    "height":      h,
                    "resolution":  f"{w}x{h}",
                    "is_high_res": w >= min_width
                })

        doc.close()

    # Write out metadata CSV
    if records:
        df       = pd.DataFrame(records)
        csv_path = os.path.join(output_dir, metadata_filename)
        df.to_csv(csv_path, index=False)
        print(f"\n✅ Extracted {len(records)} images; metadata saved to {csv_path}")
        return df
    else:
        print("⚠️ No images found in any PDF.")
        return pd.DataFrame()

# ── Example usage ────────────────────────────────────────────────────
if __name__ == "__main__":
    metadata_df = extract_images_from_pdfs(
        input_dir="submissions",
        output_dir="Extracted_images",
        min_width=1200
    )

🔧 Converting vidyalatanvi_LATE_218146_14937211_COGS 160 A1-1.docx → vidyalatanvi_LATE_218146_14937211_COGS 160 A1-1.pdf …


100%|██████████| 1/1 [00:28<00:00, 28.76s/it]



📄 Extracting images from mainayardaniel_127050_14924649_COGS 160 Le Corbusier Doc.pdf (56 pages)…


In [7]:
import os
import fitz      # PyMuPDF
from PIL import Image
import io

def compress_all_pdfs(input_dir, output_dir, dpi=100, downscale_factor=2):
    """
    Compress all PDF files in `input_dir` by rendering each page to an image,
    optionally downscaling, and reassembling into a new PDF in `output_dir`.
    """
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        if not filename.lower().endswith(".pdf"):
            continue
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)
        print(f"🔄 Compressing {filename} ...")
        try:
            doc = fitz.open(input_path)
            new_pdf = fitz.open()
            for page in doc:
                pix = page.get_pixmap(dpi=dpi)
                img = Image.frombytes("RGB", (pix.width, pix.height), pix.samples)
                new_size = (pix.width // downscale_factor, pix.height // downscale_factor)
                img = img.resize(new_size, Image.LANCZOS)
                buffer = io.BytesIO()
                img.save(buffer, format="PDF", resolution=dpi)
                buffer.seek(0)
                img_pdf = fitz.open("pdf", buffer)
                new_pdf.insert_pdf(img_pdf)
            new_pdf.save(output_path)
            new_pdf.close()
            doc.close()
            print(f"✅ Saved compressed PDF: {output_path}")
        except Exception as e:
            print(f"❌ Failed to compress {filename}: {e}")


🔄 Compressing mainayardaniel_127050_14924649_COGS 160 Le Corbusier Doc.pdf ...
✅ Saved compressed PDF: compressed_files/mainayardaniel_127050_14924649_COGS 160 Le Corbusier Doc.pdf


In [13]:
import pandas as pd
from difflib import get_close_matches

# ── 1) Load CSVs ──────────────────────────────────
meta_df   = pd.read_csv("image_metadata.csv")
roster_df = pd.read_csv("student_info.csv")

# ── 2) Extract login_id from PDF filenames ────────
meta_df["login_id"] = meta_df["pdf_file"].str.split("_").str[0]

# ── 3) Tidy roster columns ────────────────────────
roster_df = roster_df.rename(columns={
    "SIS Login ID": "login_id",
    "Student":      "student_name",
    "SIS User ID":  "pid"
})
roster_df["student_name"] = roster_df["student_name"].fillna("").astype(str).str.strip()

# ── 4) Exact-match merge ──────────────────────────
merged = pd.merge(
    meta_df,
    roster_df[["login_id","student_name","pid"]],
    on="login_id",
    how="left"
)

# ── 5) Find unmatched IDs ─────────────────────────
unmatched = merged.loc[merged["pid"].isna(), "login_id"].unique()
print("🔍 Unmatched login_ids:", unmatched)

# ── 6) Prepare fuzzy matching ─────────────────────
roster_df["norm_name"] = (
  roster_df["student_name"].str.lower().str.replace(r"[^a-z0-9]", "", regex=True)
)
roster_map = { row.norm_name: (row.login_id, row.student_name, row.pid) for row in roster_df.itertuples() }

# ── 7) Fuzzy-match suggestions ────────────────────
suggestions = {}
for uid in unmatched:
    key = str(uid).lower()
    if key in roster_map:
        suggestions[uid] = [roster_map[key]]
        continue
    hits = [roster_map[n] for n in roster_map if key in n or n in key]
    if hits:
        suggestions[uid] = hits
        continue
    best = get_close_matches(key, roster_map.keys(), n=1, cutoff=0.6)
    suggestions[uid] = [roster_map[best[0]]] if best else []

# ── 8) Auto-fill unique matches ───────────────────
for uid, matches in suggestions.items():
    if len(matches) == 1:
        _, name, pid = matches[0]
        merged.loc[merged["login_id"]==uid, "student_name"] = name
        merged.loc[merged["login_id"]==uid, "pid"]          = pid

# ── 9) Report ambiguous/no matches ───────────────
for uid, matches in suggestions.items():
    if len(matches)>1:
        print(f"⚠️ {uid!r} HAS MULTIPLE MATCHES:")
        for _, name, pid in matches:
            print(f"    {name} → {pid}")
    elif not matches:
        print(f"❌ {uid!r} HAS NO CLOSE MATCH")

# ──10) Save result ────────────────────────────────
merged.to_csv("image_metadata_with_name_pid.csv", index=False)
print("✅ Done! Check → image_metadata_with_name_pid.csv")


🔍 Unmatched login_ids: ['mainayardaniel' 'emralinolalaine' 'krukjulia' 'spavenchristine' 'khirwadkarisha' 'liangmichael' 'yangheiman' 'dasilvatheo' 'davidmatthew' 'marvanalicia' 'vidyalatanvi']

✅ Done! Check → image_metadata_with_name_pid.csv
